<a href="https://colab.research.google.com/github/nceder/qpb4e/blob/main/code/Chapter%2023/Chapter_23.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 23 Saving Data

# 23.2 SQLite: Using the sqlite3 database

In [ ]:
import sqlite3

conn = sqlite3.connect("datafile.db") # Definiamo una connessione - creiamo un file se non esiste già

In [ ]:
cursor = conn.cursor() # Prima di eseguire query dobbiamo inizializzare il cursore rispetto la connessione al DB
cursor

In [ ]:
cursor.execute("create table people (id integer primary key, name text, count integer)") # Crea una tabella chiamata people con tre colonne
cursor.execute("insert into people (name, count) values ('Bob', 1)") # Inserisce una riga con nome "Bob" e count=1 (METODO 1)
cursor.execute("insert into people (name, count) values (?, ?)", # Altro inserimento, questa volta usando i parametri (?). È la forma più sicura per inserire dati dinamici (evita SQL injection). (METODO 2)
               ("Jill", 15))
conn.commit() # Per eseguire le modifiche al DB dobbiamo effettuare un COMMIT altrimenti non si salva una volto terminata la sessione.

In [5]:
# Inserisce nella tabella people una riga con name='Joe' e count=10.
# Usa i parametri nominati (:username, :usercount). Nel secondo argomento passi un dizionario che abbina i valori ai nomi. 
# Questo modo è chiaro soprattutto se hai query lunghe e molti parametri.
cursor.execute("insert into people (name, count) values (:username, :usercount)",
                  {"username": "Joe", "usercount": 10})

In [6]:
# Recupera tutte le righe dalla tabella people come LISTA di TUPLE.
result = cursor.execute("select * from people")
print(result.fetchall())

[(1, 'Bob', 1), (2, 'Jill', 15), (3, 'Joe', 10)]


In [7]:
result = cursor.execute("select * from people where name like :name",
                        {"name": "bob"})
print(result.fetchall())

[(1, 'Bob', 1)]


In [8]:
cursor.execute("update people set count=? where name=?", (20, "Jill"))
result = cursor.execute("select * from people")
print(result.fetchall())

[(1, 'Bob', 1), (2, 'Jill', 20), (3, 'Joe', 10)]


In [9]:
# Invece di usare fetchall() puoi iterare direttamente sui risultati (utile per tabelle grandi).
result = cursor.execute("select * from people")
for row in result:
    print(row)


(1, 'Bob', 1)
(2, 'Jill', 20)
(3, 'Joe', 10)


In [10]:
cursor.execute("update people set count=? where name=?", (20, "Jill"))
conn.commit()
conn.close()

### TRY This: Creating and modifying Tables
Using sqlite3, write the code that creates a database table for the Illinois weather data you loaded from a flat file in section 22.2. Suppose that you have similar data for more states and want to store more information about the states themselves. How could you modify your database to use a related table to store the state information?

In [11]:
%%writefile temp_data_01.csv
"Notes","State","State Code","Month Day, Year","Month Day, Year Code",Avg Daily Max Air Temperature (F),Record Count for Daily Max Air Temp (F),Min Temp for Daily Max Air Temp (F),Max Temp for Daily Max Air Temp (F),Avg Daily Max Heat Index (F),Record Count for Daily Max Heat Index (F),Min for Daily Max Heat Index (F),Max for Daily Max Heat Index (F),Daily Max Heat Index (F) % Coverage

,"Illinois","17","Jan 01, 1979","1979/01/01",17.48,994,6.00,30.50,Missing,0,Missing,Missing,0.00%
,"Illinois","17","Jan 02, 1979","1979/01/02",4.64,994,-6.40,15.80,Missing,0,Missing,Missing,0.00%
,"Illinois","17","Jan 03, 1979","1979/01/03",11.05,994,-0.70,24.70,Missing,0,Missing,Missing,0.00%
,"Illinois","17","Jan 04, 1979","1979/01/04",9.51,994,0.20,27.60,Missing,0,Missing,Missing,0.00%
,"Illinois","17","May 15, 1979","1979/05/15",68.42,994,61.00,75.10,Missing,0,Missing,Missing,0.00%
,"Illinois","17","May 16, 1979","1979/05/16",70.29,994,63.40,73.50,Missing,0,Missing,Missing,0.00%
,"Illinois","17","May 17, 1979","1979/05/17",75.34,994,64.00,80.50,82.60,2,82.40,82.80,0.20%
,"Illinois","17","May 18, 1979","1979/05/18",79.13,994,75.50,82.10,81.42,349,80.20,83.40,35.11%
,"Illinois","17","May 19, 1979","1979/05/19",74.94,994,66.90,83.10,82.87,78,81.60,85.20,7.85%


Writing temp_data_01.csv


In [ ]:
import sqlite3
conn = sqlite3.connect("datafile.db")

cursor = conn.cursor()


# elimina tabella nel caso fosse già stata creata
cursor.execute("DROP TABLE IF EXISTS weather")
conn.commit()  # conferma le modifiche

cursor.execute("""create table weather (id integer primary key,
              state text, state_code text,
              year_text text, year_code text, avg_max_temp real,  max_temp_count integer,
              max_temp_low real, max_temp_high real,
              heat_index real, heat_index_count integer,
              heat_index_low real, heat_index_high real,
              heat_index_coverage text)
              """)
conn.commit()


# load data from file
import csv
results = [fields[1:] for fields in csv.reader(open("temp_data_01.csv", newline=''))]
# write to database
for row in results:
  if row and row[0] != 'State':
      cursor.execute("""insert into weather (state, state_code,
              year_text, year_code, avg_max_temp,  max_temp_count,
              max_temp_low, max_temp_high,
              heat_index, heat_index_count,
              heat_index_low, heat_index_high,
              heat_index_coverage) values(?,?,?,?,?,?,?,?,?,?,?,?,?)""", row)
conn.commit()

In [29]:
result = cursor.execute("select * from weather")
print(result.fetchall())

[(1, 'Illinois', '17', 'Jan 01, 1979', '1979/01/01', 17.48, 994, 6.0, 30.5, 'Missing', 0, 'Missing', 'Missing', '0.00%'), (2, 'Illinois', '17', 'Jan 02, 1979', '1979/01/02', 4.64, 994, -6.4, 15.8, 'Missing', 0, 'Missing', 'Missing', '0.00%'), (3, 'Illinois', '17', 'Jan 03, 1979', '1979/01/03', 11.05, 994, -0.7, 24.7, 'Missing', 0, 'Missing', 'Missing', '0.00%'), (4, 'Illinois', '17', 'Jan 04, 1979', '1979/01/04', 9.51, 994, 0.2, 27.6, 'Missing', 0, 'Missing', 'Missing', '0.00%'), (5, 'Illinois', '17', 'May 15, 1979', '1979/05/15', 68.42, 994, 61.0, 75.1, 'Missing', 0, 'Missing', 'Missing', '0.00%'), (6, 'Illinois', '17', 'May 16, 1979', '1979/05/16', 70.29, 994, 63.4, 73.5, 'Missing', 0, 'Missing', 'Missing', '0.00%'), (7, 'Illinois', '17', 'May 17, 1979', '1979/05/17', 75.34, 994, 64.0, 80.5, 82.6, 2, 82.4, 82.8, '0.20%'), (8, 'Illinois', '17', 'May 18, 1979', '1979/05/18', 79.13, 994, 75.5, 82.1, 81.42, 349, 80.2, 83.4, '35.11%'), (9, 'Illinois', '17', 'May 19, 1979', '1979/05/19', 7

# 23.4 Making database handling easier with an ORM

## 23.4.1 SQLAlchemy

In [ ]:
from sqlalchemy import create_engine, select, MetaData, Table, Column, Integer, String
from sqlalchemy.orm import sessionmaker

In [ ]:
dbPath = 'datafile2.db'
engine = create_engine('sqlite:///%s' % dbPath)
metadata = MetaData()
people  = Table('people', metadata,
                Column('id', Integer, primary_key=True),
                Column('name', String),
                Column('count', Integer),
               )
Session = sessionmaker(bind=engine)
session = Session()
metadata.create_all(engine)

In [ ]:
people_ins = people.insert().values(name='Bob', count=1)
str(people_ins)

'INSERT INTO people (name, count) VALUES (:name, :count)'

In [ ]:
session.execute(people_ins)

In [ ]:
session.commit()

In [ ]:
session.execute(people_ins, [
    {'name': 'Jill', 'count':15},
    {'name': 'Joe', 'count':10}
])

In [ ]:
session.commit()
people_query = select(people)
result = session.execute(people_query)
for row in result:
    print(row)


(1, 'Bob', 1)
(2, 'Jill', 15)
(3, 'Joe', 10)


In [ ]:
result = session.execute(select(people).where(people.c.name == 'Jill'))
for row in result:
    print(row)


(2, 'Jill', 15)


In [ ]:
result = session.execute(people.update().values(count=20).where (people.c.name == 'Jill'))
session.commit()
result = session.execute(select(people).where(people.c.name == 'Jill'))
for row in result:
    print(row)


(2, 'Jill', 20)


### Mapping table objects to classes

In [ ]:
from sqlalchemy.orm import declarative_base
Base = declarative_base()
class People(Base):
    __tablename__ = "people"
    id = Column(Integer, primary_key=True)
    name = Column(String)
    count = Column(Integer)

results = session.query(People).filter_by(name='Jill')
for person in results:
    print(person.id, person.name, person.count)


2 Jill 20


In [ ]:
new_person = People(name='Jane', count=5)
session.add(new_person)
session.commit()
results = session.query(People).all()
for person in results:
    print(person.id, person.name, person.count)


1 Bob 1
2 Jill 20
3 Joe 10
4 Jane 5


In [ ]:
jill = session.query(People).filter_by(name='Jill').first()
jill.name

'Jill'

In [ ]:
jill.count = 22
session.add(jill)
session.commit()
results = session.query(People).all()
for person in results:
    print(person.id, person.name, person.count)


1 Bob 1
2 Jill 22
3 Joe 10
4 Jane 5


In [ ]:
jane = session.query(People).filter_by(name='Jane').first()
session.delete(jane)
session.commit()
jane = session.query(People).filter_by(name='Jane').first()
print(jane)

None


### Try This: Using an ORM
Using the database from earlier, write an SQLAlchemy class to map to the data table, and use it to read the records from the table.

In [ ]:
# @title
from sqlalchemy import create_engine, select, MetaData, Table, Column, Integer, String, Float
from sqlalchemy.orm import sessionmaker
dbPath = 'datafile.db'
engine = create_engine('sqlite:///%s' % dbPath)
metadata = MetaData()
Session = sessionmaker(bind=engine)
session = Session()
metadata.create_all(engine)
weather  = Table('weather', metadata,
                Column('id', Integer, primary_key=True),
                Column("state", String),
                Column("state_code", String),
                Column("year_text", String ),
                Column("year_code", String),
                Column("avg_max_temp", Float),
                Column("max_temp_count", Integer),
                Column("max_temp_low", Float),
                Column("max_temp_high", Float),
                Column("heat_index", Float),
                Column("heat_index_count", Integer),
                Column("heat_index_low", Float),
                Column("heat_index_high", Float),
                Column("heat_index_coverage", String)
                )
Session = sessionmaker(bind=engine)
session = Session()
result = session.query(weather).all()
for row in result:
    print(row)

## 23.4.2 Using Alembic for database schema changes

In [ ]:
! rm -rf alembic/

In [ ]:
! pip install alembic
! alembic init alembic

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.2/233.2 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 5.9 MB/s eta 0:00:00
Creating directory '/content/alembic' ...  done
Creating directory '/content/alembic/versions' ...  done
Generating /content/alembic/env.py ...  done
Generating /content/alembic/script.py.mako ...  done
Generating /content/alembic/README ...  done
Generating /content/alembic.ini ...  done
Please edit configuration/connection/logging settings in '/content/alembic.ini' before proceeding.


In [ ]:
# This cell will update the alembic.ini file

! sed -i 's/driver:\/\/user:pass@localhost\/dbname/sqlite:\/\/\/datafile.db/' alembic.ini

In [ ]:
# This cell creates the first revision script
result = ! alembic revision -m "create an address table"
filename= result[0].replace('Generating ', "").replace(" ...  done","")
version = filename.split("/")[-1].split("_")[0]

In [ ]:
# This cell updates the revision script's upgrade() and downgrade() functions
upgrade_cmd = """def upgrade() -> None:
    op.create_table(
        'address',
        sa.Column('id', sa.Integer, primary_key=True),
        sa.Column('address', sa.String(50), nullable=False),
        sa.Column('city', sa.String(50), nullable=False),
        sa.Column('state', sa.String(20), nullable=False),
    )
"""
downgrade_cmd = """def downgrade() -> None:
    op.drop_table('address')"""

version_file = open(filename).read()
version_file = version_file.replace("""def upgrade() -> None:
    pass""", upgrade_cmd)
version_file = version_file.replace("""def downgrade() -> None:
    pass""", downgrade_cmd)
print(version_file)

open(filename, "w").write(version_file)

"""create an address table

Revision ID: def74d4e1932
Revises: 
Create Date: 2024-09-27 02:02:49.374125

"""
from typing import Sequence, Union

from alembic import op
import sqlalchemy as sa


# revision identifiers, used by Alembic.
revision: str = 'def74d4e1932'
down_revision: Union[str, None] = None
branch_labels: Union[str, Sequence[str], None] = None
depends_on: Union[str, Sequence[str], None] = None


def upgrade() -> None:
    op.create_table(
        'address',
        sa.Column('id', sa.Integer, primary_key=True),
        sa.Column('address', sa.String(50), nullable=False),
        sa.Column('city', sa.String(50), nullable=False),
        sa.Column('state', sa.String(20), nullable=False),
    )



def downgrade() -> None:
    op.drop_table('address')



771

In [ ]:
for table in metadata.sorted_tables:
    print(table.name)

people


In [ ]:
! alembic upgrade head

INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.


In [ ]:
metadata.sorted_tables

[Table('people', MetaData(), Column('id', Integer(), table=<people>, primary_key=True, nullable=False), Column('name', String(), table=<people>), Column('count', Integer(), table=<people>), schema=None)]

In [ ]:
Session = sessionmaker(bind=engine)
session = Session()


In [ ]:
metadata.reflect(engine)
metadata.tables.keys()
session.commit()

In [ ]:
! alembic downgrade -1


INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
INFO  [alembic.runtime.migration] Running downgrade def74d4e1932 -> , create an address table


In [ ]:
for table in metadata.sorted_tables:
    print(table.name)

people


# 23.6 key:value stores with Redis

In [2]:
!pip install redis

### NOTE: You must create an account at redis.io or on another redis server for the examples to work properly.

Be sure to use the correct hostname and password for your account in cell below.

In [ ]:
import redis

r = redis.Redis(
    # set your host here
    host='',
    port=10032,
    # set your password below
    password='')

#### Basic operations

In [4]:
r.keys()

[]

#### Array operations

In [5]:
r.set('a_key', 'my value')

True

In [6]:
r.keys()

[b'a_key']

In [7]:
v = r.get('a_key')
v

b'my value'

In [8]:
r.incr('counter')

1

In [9]:
r.get('counter')

b'1'

In [10]:
r.incr('counter')

2

In [11]:
r.get('counter')

b'2'

In [12]:
r.rpush("words", "one")

1

In [13]:
r.rpush("words", "two")

2

In [14]:
r.lrange("words", 0, -1)

[b'one', b'two']

In [15]:
r.rpush("words", "three")

3

In [16]:
r.lrange("words", 0, -1)

[b'one', b'two', b'three']

In [17]:
r.llen("words")

3

In [18]:
r.lpush("words", "zero")

4

In [19]:
r.lrange("words", 0, -1)

[b'zero', b'one', b'two', b'three']

In [20]:
r.lrange("words", 2, 2)

[b'two']

In [21]:
r.lindex("words", 1)

b'one'

In [22]:
r.lindex("words", 2)

b'two'

### Expiration of values

In [23]:
r.setex("timed", 10,  "10 seconds")

True

In [24]:
r.pttl("timed")

9019

In [25]:
r.pttl("timed")

6720

In [26]:
b"timed" in r.keys()

True

### Quick Check: Uses of Key:Value stores
What sorts of data and applications would benefit most from a key:value store like Redis?

* Quick lookup of data
* Caching


# 23.7 Documents in MongoDB

In [27]:
! pip install pymongo

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 25.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.6/313.6 kB 21.8 MB/s eta 0:00:00


### NOTE: you must have an account at mongondb.net or on another Mongo server for this code to work properly.

You may also have to authorize Access from Anywhere to connect to your DB.

In [ ]:
from pymongo import MongoClient
client = MongoClient(host='*** connection string here ***')   #A

In [35]:
import datetime
a_document = {'name': 'Jane',
              'age': 34,
              'interests': ['Python', 'databases', 'statistics'],
              'date_added': datetime.datetime.now()
}
db = client.my_data     #A
collection = db.docs   #B
result = collection.find_one()  #C
db.list_collection_names()

['docs']

In [36]:
collection.insert_one(a_document)


InsertOneResult(ObjectId('674933fd4b145489056d9af8'), acknowledged=True)

In [37]:
collection.find_one()    #A

{'_id': ObjectId('674933fd4b145489056d9af8'),
 'name': 'Jane',
 'age': 34,
 'interests': ['Python', 'databases', 'statistics'],
 'date_added': datetime.datetime(2024, 11, 29, 3, 24, 32, 765000)}

In [43]:
from bson.objectid import ObjectId

###----->>>> use ObjectId from cell above in code below

collection.find_one({"_id":ObjectId('674933fd4b145489056d9af8')})  #B


{'_id': ObjectId('674933fd4b145489056d9af8'),
 'name': 'Jane',
 'age': 34,
 'interests': ['Python', 'databases', 'statistics'],
 'date_added': datetime.datetime(2024, 11, 29, 3, 24, 32, 765000)}

In [44]:
# use ObjectId from above in code below
collection.update_one({"_id":ObjectId('674933fd4b145489056d9af8')}, {"$set": {"name":"Ann"}})       #C

UpdateResult({'n': 1, 'electionId': ObjectId('7fffffff00000000000001a6'), 'opTime': {'ts': Timestamp(1732850721, 33), 't': 422}, 'nModified': 1, 'ok': 1.0, '$clusterTime': {'clusterTime': Timestamp(1732850721, 33), 'signature': {'hash': b'd{\xcc\xd7\xe5\xf3}T\xe0\x84"x\xd8\x04x+\xaa\x15T\xf7', 'keyId': 7394931654956941332}}, 'operationTime': Timestamp(1732850721, 33), 'updatedExisting': True}, acknowledged=True)

In [40]:
# use ObjectId from above in code below
collection.find_one({"_id":ObjectId('66f616d8b15e1d493ca8b8d0')})

In [45]:
# use ObjectId from above in code below
collection.replace_one({"_id":ObjectId('674933fd4b145489056d9af8')}, {"name":"Maria"})                 #D

UpdateResult({'n': 1, 'electionId': ObjectId('7fffffff00000000000001a6'), 'opTime': {'ts': Timestamp(1732850725, 34), 't': 422}, 'nModified': 1, 'ok': 1.0, '$clusterTime': {'clusterTime': Timestamp(1732850725, 34), 'signature': {'hash': b'g\x82q\xf2J\xad\xeeXG\xed\r\x8atO\xbf\xe5\xc0\x90\x9e ', 'keyId': 7394931654956941332}}, 'operationTime': Timestamp(1732850725, 34), 'updatedExisting': True}, acknowledged=True)

In [46]:
# use ObjectId from above in code below
collection.find_one({"_id":ObjectId('674933fd4b145489056d9af8')})

{'_id': ObjectId('674933fd4b145489056d9af8'), 'name': 'Maria'}

In [47]:
# use ObjectId from above in code below
collection.delete_one({"_id":ObjectId('674933fd4b145489056d9af8')}) #E

DeleteResult({'n': 1, 'electionId': ObjectId('7fffffff00000000000001a6'), 'opTime': {'ts': Timestamp(1732850731, 39), 't': 422}, 'ok': 1.0, '$clusterTime': {'clusterTime': Timestamp(1732850731, 39), 'signature': {'hash': b"\xa7\xcf7g\x89}\xfc\xa2\x97\xc44\xa9V ']j\x0f\x1fn", 'keyId': 7394931654956941332}}, 'operationTime': Timestamp(1732850731, 39)}, acknowledged=True)

In [48]:
collection.find_one()

In [49]:
db.list_collection_names()

['docs']

In [50]:
collection.drop()
db.list_collection_names()

[]

###Quick Check: Uses of MONGODB
Thinking back over the various data samples you’ve seen so far and other types of data in your experience, which do you think would be well suited to being stored in a database like MongoDB? Would others clearly not be suited, and if so, why not?

#### Discussion
Data that comes in large and/or more loosely organized chunks is suited to MongoDB, such as the contents of a web page or document.

Data with a specific structure is better suited to relational data. The weather data you've seen is a good example.

# Lab 23 Create a database

This is an open-ended challenge, so there is no “official” answer provided.